In [4]:
import PyPDF2
import pandas as pd
import re
import glob
from itertools import compress

In [40]:
rowsList = []
for pdf in glob.glob('./syllabi/*.pdf'):
    file = open(pdf, 'rb')
    fileReader = PyPDF2.PdfFileReader(file)

    text = ''
    for i in range(fileReader.numPages):
        text += fileReader.getPage(i).extractText()
    text = text.replace("\n", " ")
    while "  " in text:
        text = text.replace("  ", " ")
    while " ." in text:
        text = text.replace(" .", ".")   
            
    emails = re.findall('(?:\s|\))(\w+?@[\w.]+?\.\w+?)(?:\s|\))', text)
    links = re.findall('\s((?:(?:https?://)|(?:www\.))[\w\./~]+?)\s', text)
    time = re.findall('(?i)(?:1[0-2]|[0-9]|2[0-4]) ?: ?[0-6][0-9](?:\s?(?:p|a)m)?', text)
    timeRange = re.findall('(?i)(?:1[0-2]|[0-9]|2[0-4])(?::[0-6][0-9])?\s?-\s?(?:1[0-2]|[0-9]|2[0-4]) ?: ?[0-6][0-9](?:\s?(?:p|a)m)?', text)
    phone = re.findall('(?:\+?\d(?:-|\.)?)?(?:(?:\(\d{3}\))|(?:\d{3}))(?:-|\.)?\d{3}(?:-|\.)?\d{4}', text)
    weights = re.findall('\d{1,3}%', text)
    points = re.findall('\d{1,3} (?:points|pts)', text)
    isbn = re.findall('(?i)ISBN(?:13)?:? ([\d\-a-z]{10,})', text)
    terms = re.findall('(?i)(?:(?:Fall)|(?:Spring)|(?:Summer)|(?:Winter)) ?\d{2,4}', text)
    
    monthPattern = '(?:(?:Oct(?:(?:ober)|\.))|(?:Sep(?:(?:tember)|\.))|(?:Nov(?:(?:ember)|\.))|(?:Dec(?:(?:ember)|\.))|(?:Jan(?:(?:uary)\.))|(?:Feb(?:(?:ruary)|\.))|(?:Mar(?:(?:ch)|\.))|(?:May)|(?:June)|(?:July)|(?:Aug(?:(?:ust)|\.))|(?:Apr(?:(?:il)|\.)))'
    datePattern1 = '(?:\d{2}/\d{2}/(?:\d{2}|\d{4}))'
    datePattern2 = monthPattern + ' ?\d{2}(?:,? ?(?:\d{2}|\d{4}))?'
    dates = re.findall('(?i)(' + datePattern1 + '|' + datePattern2 +')', text)
       
    codes = re.findall('[A-Z]{2,}(?:-| )?\d{2,5}(?:[-\d\/]+)?', text)
    stopWords = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY', 'ISBN', 'FALL', 'SPRING', 'SUMMER', 'EXAM']
    codes = list(compress(codes, [not any([word in code for word in stopWords]) for code in codes]))
    
    rooms = re.findall('(?i)Room\s?\d{1,3}[a-z]?', text)
    offices = re.findall('(?:O|o)(?:F|f)(?:F|f)(?:I|i)(?:C|c)(?:E|e):?\s([A-Z][a-z]+ ?\d{1,})', text)
    locations = []
    locations += rooms
    locations += offices
    
    instructor = re.findall('(?:Instructor|Coordinator):\s([A-Z][a-z]+\s[A-Z](?:[a-z]+|\.)(?:\s[A-Z][a-z]+)?)', text)
    if len(instructor) > 0:
        while('Email' in instructor[0]):
            instructor[0] = instructor[0].replace('Email', '')
        while('Office' in instructor[0]):
            instructor[0] = instructor[0].replace('Office', '')
    
    department = re.findall('Department(?:\sof|:)\s((?:[A-Z][a-z]+\s?)+)', text)
    
#     surnames = re.findall('Mrs?\. ([A-Z][a-z]+) ', text)

    hours = re.findall('(?i)((?:hours: \d+)|(?:\d+ hours))', text)
    credits = re.findall('(?i)((?:Credits:\s?\d{1,2})|(?:[1-9]\d?\scredits?))', text)
    credits += hours
    
    years = re.findall('(?:(?:199)|(?:20(?:0|1|2)))\d', text)
    
    assignments = re.findall('(?i)Assignment', text)
    reports = re.findall('(?i)Report', text)
    presentations = re.findall('(?i)Presentation', text)
    nHW = len(assignments) + len(reports) + len(presentations)
    
    literature = re.findall('((?:[A-Z][a-z]+),?\s?-?(?:[A-Z][a-z]*\.?)+[ ,\.&]{1,3})+(\(?(?:(?:199)|(?:20(?:0|1|2)))\d\)\.?)? ([A-Z[a-z]+:?( [A-Za-z:]+)+)', text)
    
    address = re.findall('(?:\d+\s)?(?:[A-Z][a-z]+\sSt)|(?:[A-Z][a-z]+\sAve)|(?:[A-Z][a-z]+\sWay)', text)
    
    row = {'emails':str(emails), 
           'links':str(links),
           'time':str(time),
           'timeRange':str(timeRange),
           'phone':str(phone),
           'weights':str(weights),
           'points':str(points),
           'ISBNs':str(isbn),
           'terms':str(terms), 
           'dates':str(dates), 
           'credits': str(credits), 
           'codes': str(codes), 
           'locations': str(locations), 
           'instructor': str(instructor), 
           'department':str(department), 
#            'surnames': str(surnames),
           'number of HWs': str(nHW),
           'years': str(years),
           'literature': str(literature),
           'address': str(address)
          }
    rowsList.append(row)
    
features = pd.DataFrame(rowsList) 
features.insert(0, 'FileName', [re.findall('\./syllabi\\\\(.+)\.pdf', pdf)[0] for pdf in glob.glob('./syllabi/*.pdf')])

In [41]:
features

,FileName,ISBNs,address,codes,credits,dates,department,emails,instructor,links,literature,locations,number of HWs,phone,points,terms,time,timeRange,weights,years
0,10.1.1.100.8027,[],"['2 General St', 'Data St', 'Language St', 'Da...",[],"['180 hours', '90 hours', '90 hours', '90 hour...",[],"['Science Teaching ', 'Science Teaching ', 'Sc...","['galezer@cs.openu.ac.il', 'harel@wisdom.weizm...",[],[],"[('Computer Science,', '', 'The Open Universit...",[],39,[],[],[],['8:10'],[],"['75%', '25%', '30%', '50%', '50%', '25%', '25...","['1999', '1990', '1993', '1993', '1996', '1990..."
1,82445,[],"['Needed St', 'Integrity St', 'Carry St', 'Wee...","['COSC 1436', 'PC 46', 'JO 200', 'JO 200']",[],[],[],['StudentDisabilityServices@tamuc.edu'],['Thomas L. Brown'],"['http://csci.website', 'http://sourcefo', 'ht...","[('John Lewis.', '', 'Computer Science Illumin...",['Room 16'],10,"['903.886.5409', '903.886.5404', '903.468.6000...",['100 points'],['FALL 2018'],"['9:00', '11:00', '9:30am', '2:00pm', '8:00am']",[],[],"['2018', '2013', '2010', '2007']"
2,CIS_4911,[],"['Outcomes St', '5 Data St']","['CIS 4911', 'CEN 4010', 'CEN 4010']",['Credits: 3'],[],[],['clarkep@cis.fiu.edu'],['Peter Clarke'],['http://www.cis.fiu.edu/programs/undergrad/cs...,"[('Computer Science.', '', 'Subject Area Coord...",[],9,[],[],[],[],[],[],['2001']
3,Comp_Sci_T_Burton_WebV1,[],"['Conditional St', 'Iterative St', 'The St', '...",[],[],[],[],[],[],[],"[('Course Description,', '', 'and the APCS A R...",[],0,[],[],[],[],[],[],[]
4,cooks_syllabus9-29,[],['Communication St'],"['COMM 499', 'COMM 499', 'COMM 499', 'COMM 499...",[],"['October 23', 'November 13', 'November 29', '...",['Communication '],['leda@comm.umass.edu'],[],"['www.comcol.umass.edu/capstone.', 'http://www...","[('Dialogue, Discourse,', '', 'Identity and Co...",[],20,[],"['25 points', '75 points', '3 points', '110 po...","['Fall 2012', 'Fall 2012', 'Spring 2013', 'Fal...","['5:15', '2:20', '3:50', '8:30', '4:45']","['4-5:15', '2:20-3:50', '8:30-4:45']",[],"['2012', '2012', '2013', '2011', '2011', '2003..."
5,CS 132.01_ Fundamentals of Computer Science II,[],[],['CS 132'],[],[],[],[],[],[],[],[],0,[],[],[],[],[],[],[]
6,CS 63017-73017 Big Data Management,[],['Column St'],['CS 63017'],['3 Credit'],[],[],[],[],[],"[('Definitive Guide,', '', 'Tom White', ' Whit...",[],0,[],[],[],[],[],[],"['2015', '2010', '2010', '2000']"
7,CS115syl,[],[],['CS 115'],[],[],[],[],[],['www.cs.hmc.edu/csforall'],"[('Libeskind-Hadas,', '', 'CS for All', ' All')]",[],4,[],[],[],[],[],[],[]
8,CS8W18_syllabus,[],"['Overview St', 'General St', 'Disabled St', '...","['SMSS 4409', 'PHELP 3525', 'PHELP 3525', 'TOT...","['12 hours', '9 hours', '3 hours', '24 hours']",['March 21'],[],"['zmatni@ucsb.edu', 'yunzhao@umail.ucsb.edu', ...",['Ziad Matni '],"['https://piazza.com/ucsb/winter2018/cs8', 'ht...","[('Computer Science,', '', 'an abstraction is ...",[],21,"['1449699390', '1449699391']",[],"['Winter 2018', 'winter2018', 'Winter 2018']","['9:30 AM', '10:45 AM', '10:30 am', '11:30 am'...",[],"['10%', '20%', '0%', '0%', '15%', '25%', '25%'...","['2018', '2018', '2018']"
9,CSET 1100 MS,[],['Control St'],['MATH 1330'],['Hours: 1'],[],[],[],['Jared Oluoch'],[],"[('Computer Science &', '', 'Engineering Techn...",[],3,['0132747189'],[],[],['10: 01'],[],[],['2012']


In [134]:
features.to_csv('features-retrieved-by-IhorMarkevych.csv', header=False, index=False)